In [3]:
import re
import os
import xml.etree.ElementTree as ET
import mysql.connector
import pandas as pd

In [4]:
# Creación de la base de datos
def creacion_bbdd(nombrebbdd):
    cnx = mysql.connector.connect(user = 'root', password = 'AlumnaAdalab', host = 'localhost')
    mycursor = cnx.cursor()
    try:
        mycursor.execute(f"CREATE DATABASE {nombrebbdd}")
        print(mycursor)

    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)
    else:
        mycursor.close()
        cnx.close()

In [5]:
nombre_bbdd = "proyecto1_team1"
creacion_bbdd(nombre_bbdd)

1007 (HY000): Can't create database 'proyecto1_team1'; database exists
Error Code: 1007
SQLSTATE HY000
Message Can't create database 'proyecto1_team1'; database exists


In [6]:
# Creamos las tablas
def creacion_tabla(query,nombrebbdd):
    
    cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                                host='127.0.0.1', database= nombrebbdd)

    mycursor = cnx.cursor()
    
    try: 
        mycursor.execute(query)
        cnx.commit()
        print(mycursor)

    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)
    
    else:
        mycursor.close()
        cnx.close()

In [7]:
tablasql = """CREATE TABLE tabla_sql(index_sql int, q10_part_1 varchar(255),
                        q10_part_2 varchar(255), q10_part_3 varchar(255), q10_part_4 varchar(255),
                        q10_part_5 varchar(255), q10_part_6 varchar(255), q10_part_7 varchar(255),
                        q10_part_8 varchar(255), q10_part_9 varchar(255), q10_part_10 varchar(255),
                        q10_part_11 varchar(255), q10_part_12 varchar(255), q10_part_13 varchar(255),
                        q10_part_14 varchar(255), q10_part_15 varchar(255), q10_part_16 varchar(255),
                        q10_other varchar(255), d482xta varchar(255), PRIMARY KEY(index_sql))"""

In [8]:
bbdd = "proyecto1_team1"

In [9]:
creacion_tabla(tablasql,bbdd)

1050 (42S01): Table 'tabla_sql' already exists
Error Code: 1050
SQLSTATE 42S01
Message Table 'tabla_sql' already exists


In [10]:
tablaxml = """CREATE TABLE IF NOT EXISTS tabla_xml(
                        index_xml INT,
                        `time` VARCHAR(255),
                        age VARCHAR(255),
                        gender VARCHAR(255),
                        index_sql INT,
                        PRIMARY KEY(index_xml),
                        CONSTRAINT fk_sql_xml
                        FOREIGN KEY (index_sql)
                        REFERENCES tabla_sql(index_sql))"""

In [11]:
creacion_tabla(tablaxml,bbdd)

MySQLCursor: CREATE TABLE IF NOT EXISTS tabla_xml(
  ..


In [14]:
tablatxt ="""CREATE TABLE IF NOT EXISTS tabla_txt(
                        index_txt INT, index_sql INT, q3 VARCHAR(255), q4 VARCHAR(255),
                        q5 VARCHAR(255), q6 VARCHAR(255), q7 VARCHAR(255), q8 VARCHAR(255),
                        q9 VARCHAR(255), q11 VARCHAR(255), q12 VARCHAR(255), q13 VARCHAR(255),
                        q14 VARCHAR(255), q15 VARCHAR(255), q16 VARCHAR(255), q17 TEXT,
                        q20 VARCHAR(255), q21 VARCHAR(255), q22 VARCHAR(255), q23 VARCHAR(255),
                        q24 TEXT, q25 VARCHAR(255), q26 VARCHAR(255), q32 TEXT,
                        q33 VARCHAR(255), q34 VARCHAR(255), q35 VARCHAR(255), q41 VARCHAR(255),
                        PRIMARY KEY(index_txt),
                        CONSTRAINT fk_sql_txt FOREIGN KEY (index_sql) REFERENCES tabla_sql(index_sql))"""

In [15]:
creacion_tabla(tablatxt,bbdd)

MySQLCursor: CREATE TABLE IF NOT EXISTS tabla_txt(
  ..


In [12]:
class Lectura_Procesado_datos:
    def __init__(self,nombre_fichero, tipo_fichero):
        self.nombre_fichero = nombre_fichero
        self.tipo_fichero = tipo_fichero

    def lectura_fichero(self):
        if self.nombre_fichero[-4:] == ".txt":
            with open(self.nombre_fichero,'r') as file:
                self.nombre_fichero = file.readlines() 
            return self.nombre_fichero

        elif self.nombre_fichero[-4:] == ".xml":
            
            tree = ET.parse(self.nombre_fichero)
            root = tree.getroot()

            self.nombre_fichero = []
            for child in root:
                xml_dict = {}
                for subchild in child:
                    xml_dict[subchild.tag] = subchild.text
                self.nombre_fichero.append(xml_dict) # Es una lista que contiene diccionarios
            return self.nombre_fichero
    
    def limpieza_fichero(self,lista,tipo_fichero):

        if tipo_fichero == "txt":
            for i in lista:
                lista[lista.index(i)] = lista[lista.index(i)].replace("null","NULL").replace("<","under").replace("\n","").split(";")
        # Quitamos los espacios antes y después de los strings
            for i in lista:
                for n in i:
                    i[i.index(n)] = n.strip()
            contador = 0
            for i in lista:
                i.insert(1,contador)
                contador +=1
                try:
                    i[0] = int(i[0])
                except:
                    continue
            
            tupla_txt = [tuple(i) for i in lista]
            
            tupla_txt.pop(0)
            
            return tupla_txt
        
        elif tipo_fichero == "xml":
            for item in lista:
                item.pop("level_0", "No existe") # por si level_0 no existe
                item["index"] = int(item["index"])
                if item["gender"] == '0':
                    item.update({"gender": "Man"})
                elif item["gender"] == '1':
                    item.update({"gender": "Woman"})
                elif item["gender"] == '2':
                    item.update({"gender": "Nonbinary"})
                elif item["gender"] == '3':
                    item.update({"gender": "Prefer not to say"})
                elif item["gender"] == '4':
                    item.update({"gender": "Prefer to self-describe"})
                item.setdefault("index_sql", int(item["index"]))
            
            tupla_xml = []
            for i in lista:
                tupla_xml.append(tuple(i.values()))
            
            return tupla_xml

In [13]:
clase_xml = Lectura_Procesado_datos("data_xml.xml", "xml")

In [14]:
lectura_xml = clase_xml.lectura_fichero()

In [15]:
final_xml = clase_xml.limpieza_fichero(lectura_xml,"xml")

In [16]:
clase_txt = Lectura_Procesado_datos("data_txt.txt", "txt")

In [18]:
lectura_txt = clase_txt.lectura_fichero()

In [19]:
final_txt = clase_txt.limpieza_fichero(lectura_txt,"txt")

In [20]:
def insercion_datos(query,lista,nombrebbdd):
    

    cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                                host='127.0.0.1', database=nombrebbdd)


    mycursor = cnx.cursor()
    
    
    try: 
        
        mycursor.execute("SET foreign_key_checks = 0;")
        mycursor.executemany(query, lista)
        cnx.commit()
        print(mycursor.rowcount, "registro insertado.")
        mycursor.execute("SET foreign_key_checks = 1;")

    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)
    
    else:
        mycursor.close()
        cnx.close()

In [21]:
sql_xml = "INSERT INTO tabla_xml(index_xml, `time`, age, gender, index_sql) VALUES (%s,%s,%s,%s,%s);"

In [22]:
insercion_datos(sql_xml,final_xml,bbdd)

25972 registro insertado.


In [23]:
sql_txt = """ INSERT INTO tabla_txt(index_txt, index_sql, q3, q4, q5, q6, q7, q8, q9, q11, q12, q13, q14, q15, q16, q17, q20, q21, q22, q23, q24, q25, q26, q32, q33, q34, q35, q41 ) 
                                VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""

In [24]:
insercion_datos(sql_txt,final_txt,bbdd)

25972 registro insertado.
